In [4]:
import numpy as np

# Denavit-Hartenberg parameters for the robot
d = [0, 0, 0.2, 0, 0, 0]
a = [0, 0.4, 0.3, 0, 0, 0]
alpha = [np.pi/2, 0, np.pi/2, -np.pi/2, np.pi/2, 0]

def forward_kinematics(theta):
    """
    Perform forward kinematics to compute the end-effector position and orientation.
    :param theta: Joint angles [theta1, theta2, theta3, theta4, theta5, theta6]
    :return: Homogeneous transformation matrix representing the end-effector pose
    """
    T = np.eye(4)
    for i in range(6):
        A = np.array([[np.cos(theta[i]), -np.sin(theta[i])*np.cos(alpha[i]), np.sin(theta[i])*np.sin(alpha[i]), a[i]*np.cos(theta[i])],
                      [np.sin(theta[i]), np.cos(theta[i])*np.cos(alpha[i]), -np.cos(theta[i])*np.sin(alpha[i]), a[i]*np.sin(theta[i])],
                      [0, np.sin(alpha[i]), np.cos(alpha[i]), d[i]],
                      [0, 0, 0, 1]])
        T = np.dot(T, A)
    return T

def inverse_kinematics(T):
    """
    Perform inverse kinematics to compute the joint angles given the desired end-effector pose.
    :param T: Homogeneous transformation matrix representing the end-effector pose
    :return: Joint angles [theta1, theta2, theta3, theta4, theta5, theta6]
    """
    theta = np.zeros(6)
    P = T[:3, 3]
    R = T[:3, :3]
    
    # Wrist center position
    P_wc = P - 0.1 * R[:, 2]
    
    # Joint 1 angle (theta1)
    theta[0] = np.arctan2(P_wc[1], P_wc[0])
    
    # Joint 3 angle (theta3)
    l = np.linalg.norm(P_wc[:2])
    h = P_wc[2] - d[0]
    D = (l**2 + h**2 - a[1]**2 - a[2]**2) / (2 * a[1] * a[2])
    theta[2] = np.arctan2(-np.sqrt(1 - D**2), D)
    
    # Joint 2 angle (theta2)
    alpha = np.arctan2(h, l)
    beta = np.arctan2(a[2] * np.sin(theta[2]), a[1] + a[2] * np.cos(theta[2]))
    theta[1] = alpha - beta
    
    # Joint 4 angle (theta4)
    R03 = np.dot(np.linalg.inv(forward_kinematics(theta[:3]))[:3, :3], R)
    theta[3] = np.arctan2(R03[1, 2], R03[0, 2])
    
    # Joint 5 angle (theta5)
    theta[4] = np.arctan2(np.sqrt(1 - R03[2, 2]**2), R03[2, 2])
    
    # Joint 6 angle (theta6)
    theta[5] = np.arctan


In [5]:
import numpy as np

def main():
    # Define the joint angles
    theta = [np.pi/6, np.pi/4, np.pi/3, np.pi/2, np.pi/6, np.pi/4]

    # Perform forward kinematics
    end_effector_pose = forward_kinematics(theta)
    print("Forward Kinematics Result:")
    print(end_effector_pose)
    print()

    # Perform inverse kinematics
    joint_angles = inverse_kinematics(end_effector_pose)
    print("Inverse Kinematics Result:")
    print(joint_angles)

if __name__ == "__main__":
    main()

Forward Kinematics Result:
[[ 0.16892669  0.14806061  0.97444437  0.27770581]
 [-0.60957691  0.79258961 -0.01475455 -0.07060658]
 [-0.77451905 -0.59150635  0.22414387  0.57262046]
 [ 0.          0.          0.          1.        ]]



IndexError: index 3 is out of bounds for axis 0 with size 3

In [1]:
import numpy as np

# DH parameters for Motoman HC10DPT Robot
dh_parameters = [
    {'a': 0, 'alpha': 0, 'd': 0.275, 'theta': 0.1},
    {'a': 0.301, 'alpha': np.pi/2, 'd': 0, 'theta': 0.2 + np.pi/2 },
    {'a': 0.7, 'alpha': 0, 'd': 0, 'theta': 0.3+ np.pi/2},
    {'a': 0, 'alpha': np.pi/2, 'd': 0.19, 'theta': 0.4 + np.pi},
    {'a': 0.0, 'alpha': -np.pi/2, 'd': 0.5, 'theta': 0.5},
    {'a': 0, 'alpha': np.pi/2, 'd': 0.162, 'theta': 0.6}
]

def homogeneous_transform(a, alpha, d, theta):
    """
    Compute the homogeneous transformation matrix given the DH parameters.
    :param a: Link length
    :param alpha: Link twist
    :param d: Link offset
    :param theta: Joint angle
    :return: Homogeneous transformation matrix
    """
    cos_theta = np.cos(theta)
    sin_theta = np.sin(theta)
    cos_alpha = np.cos(alpha)
    sin_alpha = np.sin(alpha)

    T = np.array([
        [cos_theta, -sin_theta*cos_alpha, sin_theta*sin_alpha, a*cos_theta],
        [sin_theta, cos_theta*cos_alpha, -cos_theta*sin_alpha, a*sin_theta],
        [0, sin_alpha, cos_alpha, d],
        [0, 0, 0, 1]
    ])

    return T

def forward_kinematics(theta):
    """
    Perform forward kinematics to compute the end-effector pose.
    :param theta: Joint angles [theta1, theta2, theta3, theta4, theta5, theta6]
    :return: Homogeneous transformation matrix representing the end-effector pose
    """
    T = np.eye(4)
    for i in range(6):
        T_i = homogeneous_transform(
            dh_parameters[i]['a'],
            dh_parameters[i]['alpha'],
            dh_parameters[i]['d'],
            theta[i] + dh_parameters[i]['theta']
        )
        T = np.dot(T, T_i)
    return T

def inverse_kinematics(T):
    """
    Perform inverse kinematics to compute the joint angles.
    :param T: Homogeneous transformation matrix representing the end-effector pose
    :return: Joint angles [theta1, theta2, theta3, theta4, theta5, theta6]
    """
    theta = np.zeros(6)
    T_06 = np.linalg.inv(T)

    for i in range(5, -1, -1):
        # Extract the DH parameters for the current joint
        a = dh_parameters[i]['a']
        alpha = dh_parameters[i]['alpha']
        d = dh_parameters[i]['d']

        # Compute the wrist center position
        if i == 5:
            P_wc = T_06[:3, 3]
        else:
            P_wc = T_06[:3, 3] - d * T_06[:3, 2]

        # Compute the joint angle
        if i == 0:
            theta


In [3]:
import numpy as np

def main():
    # Define the joint angles
    theta = [0.1, np.pi/2, np.pi/2, -np.pi/2, 0.5, 0.6]

    # Perform forward kinematics
    end_effector_pose = forward_kinematics(theta)
    print("Forward Kinematics Result:")
    print(end_effector_pose)
    print()

    # Perform inverse kinematics
    joint_angles = inverse_kinematics(end_effector_pose)
    print("Inverse Kinematics Result:")
    print(joint_angles)

if __name__ == "__main__":
    main()


Forward Kinematics Result:
[[-0.89149944  0.28889478 -0.34895352  0.66375149]
 [-0.04587394  0.70875136  0.70396526  0.58194406]
 [ 0.45069317  0.64359251 -0.6185987  -0.149711  ]
 [ 0.          0.          0.          1.        ]]

Inverse Kinematics Result:
None
